# HW4 (Anawered by MinseokOh)

<b>Instructions:</b> Write your answers in the empty code cells below.

<b>Penalties:</b> You will incur penalties if:
<ul>
<li>Your code is wrong</li>
<li>Your code does not work on different data</li>
<li>Your code is unnecessarily slow (you use a for loop, or you use DataFrame.apply unnecessarily, etc)
</ul>

Start by downloading the file <i>cleaned_survey.csv</i> in the same folder as this .ipynb file. The file contains the results from the student survey.  

Then, execute the code below to load the data in a  dataframe <i>df</i>. 

In [47]:
import pandas as pd
df = pd.read_csv("cleaned_survey-1.csv", index_col=0)
df.drop(['Expert'],axis=1,inplace=True)
#df.drop(['Languages','Expert'],axis=1,inplace=True)

In [48]:
pd.set_option('display.max_columns',50)

In [49]:
df.head()

,Job,Program,ProgSkills,C,CPP,CS,Java,Python,JS,R,SQL,SAS,Excel,Tableau,Regression,Classification,Clustering,Bach_0to1,Bach_1to3,Bach_3to5,Bach_5Plus,Languages
0,0.0,MSIS,4,1,1,0.0,0,1.0,1.0,0.0,1.0,0.0,1,1,1.0,4,4,0,1,0,0,6
1,0.5,MSIS,3,1,1,0.0,1,0.0,0.0,0.0,1.0,0.0,1,0,0.0,2,2,0,0,0,1,4
2,0.0,MSIS,3,0,0,0.0,1,1.0,0.0,0.0,1.0,0.0,1,0,1.0,3,3,0,0,1,0,3
3,0.0,MSIS,3,1,0,0.0,1,1.0,0.0,1.0,1.0,0.0,1,0,1.0,2,3,0,0,0,1,5
4,0.0,MSIS,3,1,0,0.0,1,1.0,0.0,0.0,1.0,0.0,1,0,0.0,1,1,0,0,1,0,4


### Data Set description

The data set contains one row for every person who filled the survey. We use <b>person</b> and <b>student</b> interchangeably.

<ul>
<li><b>Job</b>: 0 for those students without a job, 0.5 for those students with a part-time job, and 1 for those students with a full-time job
<li><b>Program</b>: program of enrollment
<li><b>C-Regression</b>: indicates whether the student knows (1) or doesn't know (0) a certain programming language or topic
<li><b>Classification</b>: indicates the level of knowledge (1-5) on classification
<li><b>Clustering</b>: indicates the level of knowledge (1-5) on clustering
<li><b>Bach_0to1</b>: 1 if time elapsed from graduation is less than a year; 0 otherwise
<li><b>Bach_1to3</b>: 1 if time elapsed from graduation is between one and three years; 0 otherwise
<li><b>Bach_3to5</b>: 1 if time elapsed from graduation is between three and five years; 0 otherwise
<li><b>Bach_5Plus</b>: 1 if time elapsed from graduation is more than five years; 0 otherwise
</ul>

### Question 1

<p>For each programming skill level, compute:</p>
<ol>
<li><b>PropPJC2</b>:the proportion (0 to 1, computed relatively to the group size) of people that know at least one among Python and Java and their Classification knowledge has to be at least 2. 
<li><b>StdevClust</b>:the standard deviation of the Clustering knowledge (function <i>std</i>)
</ol>
<p>You should return one data frame with one row for each programming skill level and two columns named <b>PropPy_or_Java_and_Class2+</b> and <b>StdevClass</b>. Don't worry about having a Hierarchical Index for the columns.</p> 

(In Camino, pick the correct **PropPy_or_Java_and_Class2+** number for student with ProgSkills = 5 )

In [50]:
# Filter for those who know at least one among Python and Java and their Classification knowledge is at least 2
df['Py_or_Java_and_Class2+'] = ((df['Python'] == 1.0) | (df['Java'] == 1.0)) & (df['Classification'] >= 2)

In [51]:
# Group by programming skills and compute PropPJC2 and StdevClust
result = df.groupby('ProgSkills').agg({
    'Py_or_Java_and_Class2+': 'mean',  # mean will give the proportion for boolean values
    'Clustering': 'std'
}).rename(columns={
    'Py_or_Java_and_Class2+': 'PropPy_or_Java_and_Class2+',
    'Clustering': 'StdevClass'
})

In [52]:
print(result)

            PropPy_or_Java_and_Class2+  StdevClass
ProgSkills                                        
1                             0.000000    0.786796
2                             0.363636    0.687552
3                             0.655172    1.085053
4                             0.583333    1.083625
5                             0.500000    0.707107


### Question 2

For each program, how many people have a Programming Skill knowledge of less than 4? Report one row for each existing program. 

(In Camino, fill in the number of students for MBA and MSIS in a format of MBA_total/MSIS_total. For example : 10/23 )

In [53]:
# Filter rows where students have Programming Skill knowledge of less than 4
filtered_df = df[df['ProgSkills'] < 4]

In [54]:
# Group by program and count the number of people
result = filtered_df.groupby('Program').size()

In [55]:
print(result)

Program
Business Man                      1
Faculty!                          1
MBA                              13
MSIS                             30
Supply Chain Mgmt & Analytics     2
dtype: int64


### Question 3

For each Job, return the rows of those people who, within their group, know the most Languages. (In Camino, pick the correct total number of people who satisfy this condition)

In [56]:
# Compute the total number of languages each person knows
df['TotalLanguages'] = df[['C', 'CPP', 'CS', 'Java', 'Python', 'JS', 'R', 'SQL', 'SAS']].sum(axis=1)

In [57]:
# Define a function to filter rows within each group
def filter_max_languages(group):
    return group[group['TotalLanguages'] == group['TotalLanguages'].max()]

In [58]:
result = df.groupby('Job').apply(filter_max_languages)

In [59]:
# Total number of people who satisfy the condition
result.reset_index(drop=True, inplace=True)
print(result)

   Job Program  ProgSkills  C  CPP   CS  Java  Python   JS    R  SQL  SAS  \
0  0.0    MSIS           4  1    1  1.0     1     1.0  1.0  0.0  1.0  0.0   
1  0.5    MSIS           3  1    1  0.0     1     1.0  1.0  0.0  1.0  0.0   
2  0.5     MBA           3  0    0  0.0     1     1.0  1.0  1.0  1.0  1.0   
3  1.0    MSIS           3  1    1  0.0     1     1.0  1.0  1.0  1.0  0.0   
4  1.0     MBA           3  1    1  1.0     1     1.0  1.0  0.0  1.0  0.0   

   Excel  Tableau  Regression  Classification  Clustering  Bach_0to1  \
0      1        1         1.0               2           2          0   
1      1        1         0.0               2           2          0   
2      1        1         1.0               3           3          0   
3      0        1         1.0               2           2          0   
4      1        0         1.0               1           1          0   

   Bach_1to3  Bach_3to5  Bach_5Plus  Languages  Py_or_Java_and_Class2+  \
0          1          0       

In [60]:
total_people = len(result)
print(total_people)

5


### Question 4

Let us define the "data science experience" of a given person as the person's largest score among Regression, Classification, and Clustering. Compute the average data science experience among all MSIS students. 
(In Camino, pick the correct number)

In [61]:
# Compute the "data science experience" for each person
df['DS_Experience'] = df[['Regression', 'Classification', 'Clustering']].max(axis=1)

In [62]:
# Filter MSIS students and compute the average of their data science experience
average_experience_MSIS = df[df['Program'] == 'MSIS']['DS_Experience'].mean()

In [63]:
print(average_experience_MSIS)

1.875


### Question 5

Among those with at least one year elapsed from their Bachelor's degree, find out who is the "most knowledgeable" person. The most knowledgeable person is the one who knows Classification best (in case of ties, consider whether they know C, then CPP, then CS, then Java, then SAS). Retrieve that person's Program information. (pick the right one in Camino)

In [64]:
# Filter for those with at least one year elapsed from their Bachelor's degree
filtered_df = df[(df['Bach_1to3'] == 1) | (df['Bach_3to5'] == 1) | (df['Bach_5Plus'] == 1)]

In [65]:
sorted_df = filtered_df.sort_values(by=['Classification', 'C', 'CPP', 'CS', 'Java', 'SAS'], ascending=[False, False, False, False, False, False])

In [66]:
# Retrieve the Program information of the topmost person
most_knowledgeable_program = sorted_df.iloc[0]['Program']

In [67]:
print(most_knowledgeable_program)

Faculty!
